<a href="https://colab.research.google.com/github/SumeyyeSuslu/regression/blob/main/alldata_tecWorking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark
!pip install construct

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [948 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe am

In [ ]:
      #ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZzZ1`1

In [ ]:
!7z x DummyGeometryforPHDwork.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 30742711 bytes (30 MiB)

Extracting archive: DummyGeometryforPHDwork.7z
--
Path = DummyGeometryforPHDwork.7z
Type = 7z
Physical Size = 30742711
Headers Size = 1602
Method = LZMA2:24
Solid = +
Blocks = 2

  0%      9% 26 - DummyGeometry/mach0.200_aoa10. . 00_aoa10.00_beta0.00-POINT.dat                                                                          20% 35 - DummyGeometry/mach0.200_aoa6.00 . _mach0.200_aoa6.00_beta0.00.plt                                                               

In [ ]:
import os
file_paths = [] 
for root, dirs, files in os.walk("./DummyGeometry"):
    for file in files:
        if file.endswith(".plt"):
             file_paths.append(os.path.join(root, file))
print(file_paths)

['./DummyGeometry/mach1.000_aoa4.00_beta0.00/Tecplot_mach1.000_aoa4.00_beta0.00.plt', './DummyGeometry/mach0.200_aoa4.00_beta0.00/Tecplot_mach0.200_aoa4.00_beta0.00.plt', './DummyGeometry/mach1.000_aoa10.00_beta0.00/Tecplot_mach1.000_aoa10.00_beta0.00.plt', './DummyGeometry/mach0.800_aoa8.00_beta0.00/Tecplot_mach0.800_aoa8.00_beta0.00.plt', './DummyGeometry/mach1.000_aoa0.00_beta0.00/Tecplot_mach1.000_aoa0.00_beta0.00.plt', './DummyGeometry/mach0.600_aoa10.00_beta0.00/Tecplot_mach0.600_aoa10.00_beta0.00.plt', './DummyGeometry/mach0.600_aoa8.00_beta0.00/Tecplot_mach0.600_aoa8.00_beta0.00.plt', './DummyGeometry/mach0.800_aoa10.00_beta0.00/Tecplot_mach0.800_aoa10.00_beta0.00.plt', './DummyGeometry/mach0.200_aoa0.00_beta0.00/Tecplot_mach0.200_aoa0.00_beta0.00.plt', './DummyGeometry/mach0.200_aoa10.00_beta0.00/Tecplot_mach0.200_aoa10.00_beta0.00.plt', './DummyGeometry/mach0.200_aoa6.00_beta0.00/Tecplot_mach0.200_aoa6.00_beta0.00.plt', './DummyGeometry/mach0.200_aoa8.00_beta0.00/Tecplot_mach

In [ ]:
# Spark Session, Pipeline, Functions, and Metrics
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import rand
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import Vector
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [ ]:

import re
import numpy as np
import pltreader as plt

dataset = []
for item in file_paths:
  file_content={}
  pat = re.compile("mach(.*)_aoa(.*)_beta(.*)/")
  obj = pat.search(item)
  mach = float(obj.group(1))
  aoa = float(obj.group(2))
  beta = float(obj.group(3))
  with open( item, 'rb') as f:
    bytelist = f.read()
  header = plt.read_header(bytelist)
  data = plt.read_data(bytelist,header)
  for zone_id in range (len(data['Zones'])):
    zone_shape = data['Zones'][zone_id]['Pressure'].shape
    data['Zones'][zone_id]['mach']=mach*np.ones(zone_shape, dtype = 'float32')
    data['Zones'][zone_id]['aoa']=aoa*np.ones(zone_shape, dtype = 'float32')
    data['Zones'][zone_id]['beta']=beta*np.ones(zone_shape, dtype = 'float32')


  file_content['data'] = data
  dataset.append(file_content)

In [ ]:
cols=['mach','aoa','beta']+header['VarNames']
m=[]
for fid in range(len(file_paths)):
  t=[]
  for zid in range (len(data['Zones'])):
    for col in cols:
      t.append(dataset[fid]['data']['Zones'][zid][col])
  m.append(t)
s=np.hstack(m).transpose()

/usr/local/lib/python3.7/dist-packages/numpy/core/shape_base.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


In [ ]:
cols=['mach','aoa','beta']+header['VarNames']

s={}
for col in cols:
  t=[]
  for fid in range(len(file_paths)):
    for zid in range(len(data['Zones'])):
      t.append(dataset[fid]['data']['Zones'][zid][col])
  s[col] = np.hstack(t)


In [ ]:
import pandas as pd
pandasDF=pd.DataFrame.from_dict(s)
df=spark.createDataFrame(pandasDF) 


In [ ]:
# dataset[file_id]['data']['Zones'][zone_id][var_name])

In [ ]:
df.limit(15).toPandas()

,mach,aoa,beta,CoordinateX,CoordinateY,CoordinateZ,X Component Wall Shear,Pressure,Pressure Coefficient,Y Component Wall Shear,Z Component Wall Shear
0,1.0,4.0,0.0,11.741610,-1.658558,0.277781,247.621735,8324.561523,0.117451,-7.759322,44.087044
1,1.0,4.0,0.0,11.738341,-1.697367,0.270796,289.505920,2006.861694,0.028315,-21.355097,6.275712
2,1.0,4.0,0.0,11.705114,-1.663811,0.269339,276.453400,4227.937500,0.059652,-25.645638,6.516389
3,1.0,4.0,0.0,11.774469,-1.584260,0.253532,196.213272,-27076.242188,-0.382018,17.835201,-6.101009
4,1.0,4.0,0.0,11.839085,-1.553269,0.251952,190.492523,-29019.902344,-0.409441,17.709211,2.667213
5,1.0,4.0,0.0,11.764987,-1.520234,0.249566,196.534409,-28161.087891,-0.397324,18.178757,-1.299893
6,1.0,4.0,0.0,11.745173,-1.656447,0.262674,243.907288,-20430.775391,-0.288257,6.527334,-27.109739
7,1.0,4.0,0.0,11.790966,-1.668062,0.260443,216.297409,-24295.388672,-0.342783,13.328602,-15.915632
8,1.0,4.0,0.0,11.768639,-1.693918,0.265840,251.172501,-19591.843750,-0.276421,4.991435,-31.002720
9,1.0,4.0,0.0,11.844545,-1.690250,0.259992,214.832123,-25959.818359,-0.366266,19.990547,-10.850661


In [ ]:
assembler= VectorAssembler(
    inputCols=['mach','aoa','CoordinateX', 'CoordinateY','CoordinateZ' ],outputCol='features')


output= assembler.transform(df)
output.show

<bound method DataFrame.show of DataFrame[mach: double, aoa: double, beta: double, CoordinateX: double, CoordinateY: double, CoordinateZ: double, X Component Wall Shear: double, Pressure: double, Pressure Coefficient: double, Y Component Wall Shear: double, Z Component Wall Shear: double, features: vector]>

In [ ]:
#final data consist of features and label which is medv.
final_data=output.select('features','Pressure')
#splitting data into train and test
train_data,test_data=final_data.randomSplit([0.8,0.2])
#creating an object of class LinearRegression
#object takes features and label as input arguments
#rf = GBTRegressor(featuresCol='features',labelCol='Pressure',maxiter=10)
rf = GBTRegressor(featuresCol='features',labelCol='Pressure',maxIter=10)

#lr=LinearRegression(featuresCol='features',labelCol='Pressure')

In [ ]:

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[ rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(train_data)

# Make predictions.
predictions = model.transform(test_data)

# Select example rows to display.
predictions.select("prediction", "Pressure", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="Pressure", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[0]
print(rfModel)  # summary only

+------------------+------------------+--------------------+
|        prediction|          Pressure|            features|
+------------------+------------------+--------------------+
| 12041.75208617832|   22354.822265625|       (5,[0],[1.0])|
| 908.5088334256759| 690.4427490234375|[0.20000000298023...|
| 908.5088334256759| 646.4136962890625|[0.20000000298023...|
|30.287954763399497|415.99700927734375|[0.20000000298023...|
| 745.2516370578298| 488.6413269042969|[0.20000000298023...|
+------------------+------------------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 8997.09
GBTRegressionModel: uid=GBTRegressor_0ec7ebf71153, numTrees=10, numFeatures=5
